In [1]:
import pptk
from numpy.linalg import norm
import numpy as np
import pcl
import matplotlib.pyplot as plt
from scipy.stats import linregress
from scipy.spatial import ConvexHull
from matplotlib.path import Path
from sklearn.linear_model import RANSACRegressor

In [2]:
from sklearn.cluster import DBSCAN

In [3]:
def link(p1,p2, link_intensity):
    return np.linspace(p1, p2, num=int(norm(p1-p2)*link_intensity))

In [4]:
def convex_hull(pcl, intensity=150):
    
    hull = ConvexHull(pcl[:,:2])
    hull_path = Path( pcl[:,:2][hull.vertices] )
    results = np.vstack(hull_path.to_polygons())[:-1]
    n = results.shape[0]
    
    z_min, z_max = pcl[:,2].min(), pcl[:,2].max()
    down_points = np.hstack((results, z_min*np.ones(shape=(n,1))))
    up_points = np.hstack((results, z_max*np.ones(shape=(n,1))))
    
    box = np.zeros((0,3))

    for k in range(n):
        box = np.vstack((box, link(down_points[k%n], down_points[(k+1)%n], link_intensity=intensity)))
    for k in range(n):
        box = np.vstack((box, link(up_points[k%n], up_points[(k+1)%n], link_intensity=intensity)))
    for k in range(n):
        box = np.vstack((box, link(down_points[k%n], up_points[k%n], link_intensity=intensity)))
    
    return box

In [5]:
def load_from_bin(bin_path):
    obj = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)
    return obj[:,:3]

In [6]:
def process(pcl):
    outer_radius = 50
    inner_radius = 2.74
    
    coords = pcl.to_array()
    coords = coords[(norm(coords[:,:2], axis=1)<=outer_radius)]
    coords = coords[(norm(coords, axis=1)>inner_radius)]
    x, y, z = coords[:,0], coords[:,1], coords[:,2]
    
    ptf = coords[z<-1.3]
    reg = RANSACRegressor()
    reg.fit(ptf[:,:2],ptf[:,2])
    preds = (reg.predict(ptf[:,:2])-ptf[:,2])
    std_z = preds.std()
    mean_z = reg.predict(ptf[:,:2]).mean()
    
    to_cluster = coords[z>mean_z+1.9*std_z]
    clst = DBSCAN(eps=0.3)
    clst.fit(to_cluster)
    clst_labels = clst.labels_
    labels, counts = np.unique(clst_labels, return_counts=True)
    
    to_cluster = to_cluster[np.isin(clst_labels, labels[counts>2])]
    clst_labels = clst_labels[np.isin(clst_labels, labels[counts>2])]
    
    bhulls = np.vstack([convex_hull(to_cluster[clst_labels==k], intensity=50) for k in np.unique(clst_labels)[1:]])
    to_cluster = np.vstack((to_cluster, bhulls))
    bhull_color = int(np.quantile(np.unique(clst_labels)[1:], .50))
    clst_labels = np.hstack((clst_labels, bhull_color*np.ones(bhulls.shape[0])))
    
    return to_cluster, clst_labels

In [36]:
%time
import glob
lidar_points = glob.glob('data_2/*.pcd')
for i,p in enumerate(sorted(lidar_points)[87:]):
    pcd = pcl.load(p)
    v = pptk.viewer(*process(pcd))
    v.set(lookat=[0,0,0])
    v.set(phi=np.pi, theta=np.pi/2, r=50)
    v.capture('output/%d.png'%(i+87))

CPU times: user 10 µs, sys: 0 ns, total: 10 µs
Wall time: 18.1 µs


In [8]:
i = 30
pcd = pcl.load('data_2/00000000%d.pcd'%i)
v = pptk.viewer(*process(pcd))
v.set(lookat=[0,0,0])
v.set(phi=np.pi, theta=np.pi/2, r=50)
v.capture('output/%d.png'%(i))

In [33]:
from time import clock
t1 = time.perf_counter()
for k in range(10):
    process(pcd)
t2 = time.perf_counter() - t1

print('Wall time: %f'%(t2/10))

Wall time: 3.854762


In [32]:
import numpy as np
import glob
import sys
sys.path.remove(sys.path[2])
import cv2

In [57]:
import os
img_array = []
for filename in sorted(glob.glob('output/*.png'), key=os.path.getmtime):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)

In [58]:
out = cv2.VideoWriter('project.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()